<a href="https://www.kaggle.com/code/ssinvincible/drugs-prescription-using-reviews?scriptVersionId=170259134" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import string

import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets
from ipywidgets import interact

plt.rcParams['figure.figsize'] = (15,5)
plt.style.use('fivethirtyeight')

# Loading the dataset

In [3]:
df = pd.read_csv('/kaggle/input/drugs-dataset/drug.csv')

In [4]:
df.shape

(161297, 7)

In [5]:
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


We can say that the usefulCount is the most important dataset for  us. As this will help us the most in prescribing someone a medicine.

In [6]:
df.isna().sum()

uniqueID         0
drugName         0
condition      899
review           0
rating           0
date             0
usefulCount      0
dtype: int64

In [7]:
df.dtypes

uniqueID        int64
drugName       object
condition      object
review         object
rating          int64
date           object
usefulCount     int64
dtype: object

In [8]:
#Exploring some of the important columns in the dataset.

print("Number of Unique Drugs present in the dataset : ", df["drugName"].nunique())
print("Number of Unique Medical Conditions present in the Dataset : ", df["condition"].nunique())

print("\nThe Time Period of Collecting the data: ")
print("Starting Date: " , df["date"].min())
print("Ending Date :" , df['date'].max())

Number of Unique Drugs present in the dataset :  3436
Number of Unique Medical Conditions present in the Dataset :  884

The Time Period of Collecting the data: 
Starting Date:  1-Apr-08
Ending Date : 9-Sep-17


TASK OF PROJECT:::

1. Provide a sophisticated and useful model using NLP or ML techniques.

2. Finding most useful drugs for each condition.

3. Finding some of the hidden trends or patterns that could help the company to make precise data driven decisions.

# SUMMARIZING THE DATA

In [9]:
df.describe()

,uniqueID,rating,usefulCount
count,161297.000000,161297.000000,161297.000000
mean,115923.585305,6.994377,28.004755
std,67004.445170,3.272329,36.403742
min,2.000000,1.000000,0.000000
25%,58063.000000,5.000000,6.000000
50%,115744.000000,8.000000,16.000000
75%,173776.000000,10.000000,36.000000
max,232291.000000,10.000000,1291.000000


In Rating

50th percentile = 8 
That means most of the reviews are positive.

75th percentile = 10
That means majority people(top 25%) were pretty happy and gave 10 ratings.

In UsefulCount

Max count = 1291
min = 0
mean = 28
50% = 16

this means max count is very high compared to the mean.
Also a standard deviaation of 36 is very high.

Top 5 values with max count---

In [10]:
df.sort_values(by = 'usefulCount', ascending = False).head(4)

,uniqueID,drugName,condition,review,rating,date,usefulCount
6716,96616,Sertraline,Depression,"""I remember reading people&#039;s opinions, on...",10,31-Jul-08,1291
33552,119152,Zoloft,Depression,"""I remember reading people&#039;s opinions, on...",10,31-Jul-08,1291
21708,131116,Levonorgestrel,Birth Control,"""I have had my IUD for over a year now and I t...",10,1-Apr-09,1247
4249,182560,Mirena,Birth Control,"""I have had my IUD for over a year now and I t...",10,1-Apr-09,1247


**USELESS DRUGS ANALYSIS(usefulCount = 0)**

In [11]:
#Checking number and name of drugs wuth 0 useful count.
#This is important in analyzing why people didnt find these drugs useful.

print("Analysis on Useless Drugs:")
print("----------------------------")
print("Number of drugs with no useful count : ", df[df['usefulCount'] == 0]['uniqueID'].count())
print("Total drugs were : ", df.shape[0])

#Checking drugs with usefulCount = 0 and also rating >= 8
print("Number of good rated drugs with No useful Count: ",df.loc[(df.rating >= 8) & (df.usefulCount == 0)]['uniqueID'].count())

#Now checking average ratings of users for useless drugs
print("Average rating of useless drugs: ",df[df['usefulCount'] == 0]['rating'].mean())



Analysis on Useless Drugs:
----------------------------
Number of drugs with no useful count :  6318
Total drugs were :  161297
Number of good rated drugs with No useful Count:  2633
Average rating of useless drugs:  5.799778410889522


In [12]:
#Analysis of useful drugs
print("Analysis of useful drugs")
print("--------------------------")
print("The number of drugs with greater than 700 useful counts: ",df[df['usefulCount'] > 700].count()[0])
print("Average rating of useful drugs: ", df[df['usefulCount'] > 700]["rating"].mean())
print("Name of these drugs\n :" , df[df['usefulCount'] > 700][['drugName','condition']])

Analysis of useful drugs
--------------------------
The number of drugs with greater than 700 useful counts:  10
Average rating of useful drugs:  9.5
Name of these drugs
 :               drugName      condition
2039        Citalopram     Depression
4249            Mirena  Birth Control
5218          Implanon  Birth Control
6716        Sertraline     Depression
16889         Adipex-P    Weight Loss
21708   Levonorgestrel  Birth Control
33552           Zoloft     Depression
58608      Phentermine    Weight Loss
146145          Zoloft     Depression
152838          Celexa     Depression


/tmp/ipykernel_18/1508882189.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("The number of drugs with greater than 700 useful counts: ",df[df['usefulCount'] > 700].count()[0])


Summarizing for Categorical Data as well

In [13]:
#Lets summarize Categorical data also
df[['drugName','condition','review']].describe(include = 'object')

,drugName,condition,review
count,161297,160398,161297
unique,3436,884,112329
top,Levonorgestrel,Birth Control,"""Good"""
freq,3657,28788,33


In [14]:
df.isna().sum()

uniqueID         0
drugName         0
condition      899
review           0
rating           0
date             0
usefulCount      0
dtype: int64

In [15]:
#As missing values present in condition so lets drop them.
df = df.dropna()

In [16]:
df.isna().sum()

uniqueID       0
drugName       0
condition      0
review         0
rating         0
date           0
usefulCount    0
dtype: int64

Now there are no missing values in the dataset